In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/netflix-tv-shows-and-movies/credits.csv
/kaggle/input/netflix-tv-shows-and-movies/titles.csv


In [2]:
#loading the credits and titles datasets
df_credits= pd.read_csv('/kaggle/input/netflix-tv-shows-and-movies/credits.csv')
df_titles= pd.read_csv('/kaggle/input/netflix-tv-shows-and-movies/titles.csv')

In [3]:
df_c= df_credits.copy()
df_t= df_titles.copy()

In [4]:
#taking a quick look at the table
df_c.head()

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR


In [5]:
#taking a quick look at the table
df_t.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,48,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,113,"['crime', 'drama']",['US'],NaN,tt0075314,8.3,795222.0,27.612,8.2
2,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['comedy', 'fantasy']",['GB'],NaN,tt0071853,8.2,530877.0,18.216,7.8
3,tm70993,Life of Brian,MOVIE,"Brian Cohen is an average young Jewish man, bu...",1979,R,94,['comedy'],['GB'],NaN,tt0079470,8.0,392419.0,17.505,7.8
4,tm190788,The Exorcist,MOVIE,12-year-old Regan MacNeil begins to adapt an e...,1973,R,133,['horror'],['US'],NaN,tt0070047,8.1,391942.0,95.337,7.7


In [6]:
df_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77213 entries, 0 to 77212
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  77213 non-null  int64 
 1   id         77213 non-null  object
 2   name       77213 non-null  object
 3   character  67586 non-null  object
 4   role       77213 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.9+ MB


There are a substantial absence of values for the character played by the actors in some of the movies accounting for almost 10000 missing values in the 

In [7]:
#viewing the null data in the table
df_c[df_c.character.isnull()]

,person_id,id,name,character,role
36,3308,tm84618,Martin Scorsese,NaN,DIRECTOR
83,11475,tm127384,Terry Jones,NaN,DIRECTOR
84,11473,tm127384,Terry Gilliam,NaN,DIRECTOR
99,548006,tm70993,Peter Brett,NaN,ACTOR
106,11475,tm70993,Terry Jones,NaN,DIRECTOR
...,...,...,...,...,...
77192,1975318,tm1045018,Derel Isaac,NaN,ACTOR
77194,1975627,tm1045018,Chealse Howell,NaN,ACTOR
77197,363804,tm1045018,Pascal Atuma,NaN,DIRECTOR
77203,530811,tm1098060,Toyin Aimakhu-Johnson,NaN,ACTOR


The missing data doesnt seem like it would be needed in our analysis as we already have the names of the actors, so I would be dropping the column in totality.

In [8]:
#dropping the character column
df_c.drop(['character'], axis=1, inplace=True)

In [9]:
#checking the the changed dataset
df_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77213 entries, 0 to 77212
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  77213 non-null  int64 
 1   id         77213 non-null  object
 2   name       77213 non-null  object
 3   role       77213 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.4+ MB


No presence of null values in the movie dataset

In [10]:
df_c.sample(10)

,person_id,id,name,role
33579,598880,ts77474,Heo Jung-eun,ACTOR
17881,32421,ts20248,Will Kemp,ACTOR
44309,119098,ts75138,Chun Woo-hee,ACTOR
64535,1249803,tm830151,Sinyo Riza,ACTOR
51184,18290,tm893799,Alban Lenoir,ACTOR
55087,31888,tm461829,Ovidiu Niculescu,ACTOR
70400,98771,ts287328,Paul Nakauchi,ACTOR
22639,79924,ts35554,Tyga,ACTOR
65461,266658,tm493556,Makarand Mane,DIRECTOR
45292,311480,tm371741,Manuel Busquets,ACTOR


In [11]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5806 entries, 0 to 5805
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    5806 non-null   object 
 1   title                 5805 non-null   object 
 2   type                  5806 non-null   object 
 3   description           5788 non-null   object 
 4   release_year          5806 non-null   int64  
 5   age_certification     3196 non-null   object 
 6   runtime               5806 non-null   int64  
 7   genres                5806 non-null   object 
 8   production_countries  5806 non-null   object 
 9   seasons               2047 non-null   float64
 10  imdb_id               5362 non-null   object 
 11  imdb_score            5283 non-null   float64
 12  imdb_votes            5267 non-null   float64
 13  tmdb_popularity       5712 non-null   float64
 14  tmdb_score            5488 non-null   float64
dtypes: float64(5), int64(

In [12]:
df_t.sample(10)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1888,ts58155,Alias Grace,SHOW,"Based on the true story of Grace Marks, a hous...",2017,TV-MA,44,"['drama', 'thriller', 'crime']",['CA'],1.0,tt1034007,7.7,31577.0,15.686,7.4
4321,ts217045,Levius,SHOW,A young man whose parents became casualties of...,2019,NaN,25,"['animation', 'action', 'scifi']",['JP'],1.0,tt10619444,6.9,528.0,10.082,7.9
581,tm155774,Pomegranates and Myrrh,MOVIE,"A free spirited woman dancer, Kamar, finds her...",2009,NaN,95,['drama'],"['DE', 'KW', 'PS', 'FR']",NaN,tt1320282,6.3,249.0,2.841,6.7
4836,tm1082564,The Adam Project,MOVIE,"After accidentally crash-landing in 2022, time...",2022,PG-13,106,"['drama', 'action', 'comedy', 'scifi']",['US'],NaN,tt2463208,6.7,156603.0,919.599,7.0
4037,ts227412,Shaun the Sheep: Adventures from Mossy Bottom,SHOW,"Clever sheep Shaun, loyal dog Bitzer and the r...",2020,TV-Y7,6,"['comedy', 'family', 'animation']",['GB'],1.0,tt11983342,8.1,551.0,2.109,8.8
4889,tm1142224,The House,MOVIE,"""The House"" is an eccentric dark comedy direct...",2022,R,97,"['animation', 'drama', 'comedy', 'thriller']",['GB'],NaN,tt11703050,6.8,12270.0,8.347,7.0
1546,tm272854,Force 2,MOVIE,ACP Yashvardhan teams up RAW Agent KK to bring...,2016,PG,123,"['crime', 'action', 'thriller']",['IN'],NaN,tt5156746,6.2,5501.0,8.386,6.5
4346,ts252015,GAME ON: A Comedy Crossover Event,SHOW,Worlds collide in this special event featuring...,2020,TV-G,29,"['comedy', 'family']",['US'],2.0,tt12800428,5.4,213.0,1.761,7.7
3227,ts235758,Never Have I Ever,SHOW,The complicated life of a modern-day first gen...,2020,TV-14,28,"['drama', 'comedy']",['US'],4.0,tt10062292,7.8,45346.0,72.067,8.2
961,tm237143,Shrek the Musical,MOVIE,Shrek The Musical is a musical with music by J...,2013,PG-13,130,"['fantasy', 'family', 'comedy']",['US'],NaN,tt3070936,6.9,3634.0,15.817,7.3


ISSUES TO RESOLVE
* presence of lots null values in the dataset
* the countries are encased in square brackets which will need to be fixed.
* some of the columns are redundant and would need to be dropped
* presence of square brackets around the genre data which needs to be removed,and if needed in the analysis the genre values split from its conjoined state where you have some rows having multiple genres for a single movie 